In [61]:
import pandas as pd 
import numpy as np
import datetime
import Analysis
import pandas_ta as ta
from nselib import capital_market
from nselib import derivatives
import yfinance as yf

In [2]:
df = pd.read_csv('final_df.csv')

In [84]:
current_price = yf.download('^NSEI',period='1d',multi_level_index=False,ignore_tz=True).reset_index()['Close'].values[0]

[*********************100%***********************]  1 of 1 completed


In [3]:
df['Fetch_Time'] = pd.to_datetime(df['Fetch_Time'])

In [4]:
strike = df.Strike_Price.unique()

In [26]:
combination = df[['Strike_Price','Expiry_Date']].drop_duplicates()

In [23]:
df[df['Strike_Price'] == strike[5]]['Expiry_Date'].unique()

array(['2025-06-26', '2025-09-25', '2025-12-24', '2026-06-25',
       '2026-12-31', '2027-06-24', '2027-12-30', '2028-06-29',
       '2028-12-28', '2029-06-28', '2029-12-27', '2026-03-26'],
      dtype=object)

In [5]:
new = df[(df['Strike_Price'] == 24550) & (df['Expiry_Date']=='2025-05-08')]

In [ ]:
new['Date'] = new['Fetch_Time'].dt.date
chk = new[new['Date'] == datetime.date.today()][['Fetch_Time','CALLS_LTP']]
new[new['Date'] == datetime.date.today()]
chk1 = chk.resample('5min',on='Fetch_Time').agg({'CALLS_LTP':['first', 'min', 'max', 'last']}).reset_index()
chk1.columns = ['datetime', 'Open', 'Low', 'High', 'Close']
chk1['smma'] = Analysis.calculate_smma(chk1, period=7)
chk1['rsi'] = ta.rsi(chk1['Close'])
chk1['dema'] = ta.dema(chk1['Close'], length=10)
chk1['super_trend'] = ta.supertrend(chk1['High'], chk1['Low'], chk1['Close'], length=10, multiplier=3)['SUPERT_10_3.0']
chk1['super_trend_color'] = np.where(chk1['Close'] > chk1['super_trend'] , "green" , "red")
chk1['BBM'] = ta.bbands(close = chk1['Close'], length=20, std=2)['BBM_20_2.0']
chk1['changeover'] = np.insert(np.diff(np.where(chk1['super_trend_color'] == 'green' , 1 , 0)),0,0)
chk1['condition'] = np.insert(np.diff(np.where((chk1['BBM'] < chk1['smma']) & (chk1['smma'] < chk1['dema']) ,1,0)),0,0)
chk1 = chk1.iloc[np.where(((chk1['changeover'] == 1) & (chk1['condition'] == 1)))] 

/tmp/ipykernel_1569/788553029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['Date'] = new['Fetch_Time'].dt.date


In [39]:
dff = []
for i, row in combination.iterrows():
   try:
      strike = row['Strike_Price']
      expiry = row['Expiry_Date']
      new = df[(df['Strike_Price'] == strike) & (df['Expiry_Date']== expiry)].copy()
      new['Date'] = new['Fetch_Time'].dt.date
      # chk = new[new['Date'] == datetime.date.today()][['Fetch_Time','CALLS_LTP']]
      chk = new[['Fetch_Time','CALLS_LTP']]
      new[new['Date'] == datetime.date.today()]
      chk1 = chk.resample('5min',on='Fetch_Time').agg({'CALLS_LTP':['first', 'min', 'max', 'last']}).reset_index()
      chk1.columns = ['datetime', 'Open', 'Low', 'High', 'Close']
      chk1['smma'] = Analysis.calculate_smma(chk1, period=7)
      chk1['rsi'] = ta.rsi(chk1['Close'])
      chk1['dema'] = ta.dema(chk1['Close'], length=10)
      chk1['super_trend'] = ta.supertrend(chk1['High'], chk1['Low'], chk1['Close'], length=10, multiplier=3)['SUPERT_10_3.0']
      chk1['super_trend_color'] = np.where(chk1['Close'] > chk1['super_trend'] , "green" , "red")
      chk1['BBM'] = ta.bbands(close = chk1['Close'], length=20, std=2)['BBM_20_2.0']
      chk1['changeover'] = np.insert(np.diff(np.where(chk1['super_trend_color'] == 'green' , 1 , 0)),0,0)
      chk1['condition'] = np.insert(np.diff(np.where((chk1['BBM'] < chk1['smma']) & (chk1['smma'] < chk1['dema']) ,1,0)),0,0)
      chk1 = chk1.iloc[np.where(((chk1['changeover'] == 1) & (chk1['condition'] == 1)))] 
      chk1['strike'] = strike
      chk1['expiry'] = expiry
      dff.append(chk1)
   except:
      pass

In [56]:
df['Fetch_Time'].dt.date

0         2025-04-30
1         2025-04-30
2         2025-04-30
3         2025-04-30
4         2025-04-30
             ...    
545754    2025-05-02
545755    2025-05-02
545756    2025-05-02
545757    2025-05-02
545758    2025-05-02
Name: Fetch_Time, Length: 545759, dtype: object

In [58]:
df[df['Fetch_Time'].dt.date.isin(df['Fetch_Time'].dt.date.unique()[-2:])]

,Fetch_Time,Symbol,Expiry_Date,CALLS_LTP,Strike_Price,PUTS_LTP
0,2025-04-30 09:26:45,NIFTY,2025-12-24,0.00,13000,0.0
1,2025-04-30 09:26:45,NIFTY,2025-06-26,0.00,14000,0.0
2,2025-04-30 09:26:45,NIFTY,2026-12-31,0.00,14000,0.0
3,2025-04-30 09:26:45,NIFTY,2025-06-26,0.00,15000,0.0
4,2025-04-30 09:26:45,NIFTY,2025-12-24,0.00,15000,0.0
...,...,...,...,...,...,...
545754,2025-05-02 14:27:24,NIFTY,2029-06-28,0.00,29000,0.0
545755,2025-05-02 14:27:24,NIFTY,2029-12-27,0.00,29000,0.0
545756,2025-05-02 14:27:24,NIFTY,2025-06-26,10.15,30000,0.0
545757,2025-05-02 14:27:24,NIFTY,2025-12-24,67.95,30000,0.0


In [40]:
pd.concat(dff)

,datetime,Open,Low,High,Close,smma,rsi,dema,super_trend,super_trend_color,BBM,changeover,condition,strike,expiry
29,2025-04-30 11:50:00,6365.70,6365.70,6450.65,6450.65,6377.835714,100.000000,6393.782645,6381.430290,green,6369.9475,1,1,18000,2025-06-26
53,2025-04-30 13:50:00,2338.00,2338.00,2356.10,2356.10,2340.738985,76.017202,2343.814839,2339.591991,green,2338.9050,1,1,22000,2025-05-08
49,2025-04-30 13:30:00,1666.00,1666.00,1700.20,1700.20,1670.885714,100.000000,1677.305785,1672.780909,green,1667.7100,1,1,22650,2025-04-30
20,2025-04-30 11:05:00,1392.10,1392.10,1392.40,1392.40,1392.142857,100.000000,1392.199174,1392.147544,green,1392.1150,1,1,22950,2025-04-30
22,2025-04-30 11:15:00,1560.65,1560.65,1595.80,1595.80,1565.671429,100.000000,1572.269835,1566.528126,green,1562.4075,1,1,22950,2025-05-29
32,2025-04-30 12:05:00,1257.10,1257.10,1271.15,1271.15,1262.576604,57.604762,1262.579310,1248.087524,green,1258.8100,1,1,23300,2025-05-29
58,2025-04-30 14:15:00,1316.00,1316.00,1321.70,1321.70,1316.180239,57.834243,1317.791087,1306.525820,green,1315.4750,1,1,23500,2025-06-26
58,2025-04-30 14:15:00,800.55,800.55,803.95,803.95,799.735778,60.698307,802.010050,797.008101,green,798.8975,1,1,23550,2025-04-30
56,2025-04-30 14:05:00,1040.00,1040.00,1066.00,1066.00,1043.714286,100.000000,1048.595041,1045.178578,green,1041.3000,1,1,23550,2025-05-29
31,2025-04-30 12:00:00,729.10,729.10,729.10,729.10,723.955009,57.598524,724.218032,720.420689,green,722.0675,1,1,23800,2025-05-15


[*********************100%***********************]  1 of 1 completed


In [85]:
df[df['Strike_Price'] >= current_price]

,Fetch_Time,Symbol,Expiry_Date,CALLS_LTP,Strike_Price,PUTS_LTP
579,2025-04-30 09:26:45,NIFTY,2025-04-30,14.20,24350,76.00
580,2025-04-30 09:26:45,NIFTY,2025-05-08,208.55,24350,250.55
581,2025-04-30 09:26:45,NIFTY,2025-05-15,318.60,24350,339.90
582,2025-04-30 09:26:45,NIFTY,2025-05-22,0.00,24350,370.00
583,2025-04-30 09:26:45,NIFTY,2025-05-29,463.95,24350,424.55
...,...,...,...,...,...,...
545754,2025-05-02 14:27:24,NIFTY,2029-06-28,0.00,29000,0.00
545755,2025-05-02 14:27:24,NIFTY,2029-12-27,0.00,29000,0.00
545756,2025-05-02 14:27:24,NIFTY,2025-06-26,10.15,30000,0.00
545757,2025-05-02 14:27:24,NIFTY,2025-12-24,67.95,30000,0.00
